# Machine Intelligence with Deep Learning
## Importance batching for improved training of neural networks
---

In [1]:
import timeit

In [2]:
from models.resnet import ResNet18
from utils.data_utils import DataLoader
from utils.logging_utils import *

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import os
import pandas as pd

from datetime import datetime
today = datetime.today().strftime('%Y%m%d')

In [3]:
SEEDS = [10, 42, 4] # don't change!
#STRATEGIES = ['freeze', 'shuffle', 'homogeneous', 'heterogeneous', 'max_k_loss', 'min_k_loss'] # can be changed
STRATEGIES = ['max_k_loss', 'min_k_loss', 'heterogeneous_max_k_loss', 'heterogeneous_min_k_loss']
# FUTURE: heterogeneous_max_k_loss, ...min_loss

In [4]:
### Training
def train(epoch, optimizer, criterion_fn, seed, dataloader, strategy, device):
    criterion = criterion_fn()
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(dataloader.yield_batches(strategy, \
                                                  random_state=seed, use_train=True, \
                                                  criterion=criterion_fn(reduction='none'),\
                                                  device=device, num_iterations=500)): # 500: 50,000 images / 100 batch size
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataloader.yield_batches('freeze', \
                                                                               random_state=seed, use_train=True)):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    train_acc = 100.*correct/total
    train_loss /= total
    return train_acc, train_loss

### Testing
def test(epoch, best_acc, criterion_fn, seed, dataloader):
    criterion = criterion_fn()
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataloader.yield_batches('freeze', \
                                                                               random_state=seed, use_train=False)):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    # Save checkpoint.
    test_acc = 100.*correct/total
    test_loss /= total
    if test_acc > best_acc:
        best_acc = test_acc
        net.save(best_acc, epoch, seed, strategy)
    return test_acc, test_loss

In [5]:
resume = False
given_date = '20191113' #only needed if resumed from checkpoint
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
### task: classification of the following classes
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

    
### hyperparameters
test_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
num_epochs = 100 #100 # number of epochs the model gets trained
learning_rate = 0.01
momentum = 0.9
batch_size = 100
weight_decay = 5e-4

print("Begin training.")
start = timeit.default_timer()
#Logging header
length_table = 90
log_separating_line(length_table)
log_header_line("Seeds: {}".format(SEEDS), length_table)
log_header_line("Strategies: {}".format(STRATEGIES), length_table)
log_header_line("-> Resulting number of iterations: {}".format(len(SEEDS) * len(STRATEGIES)), length_table)
log_header_line("Number of iterations: {}".format(num_epochs), length_table)
log_header_line("Learning rate: {}".format(learning_rate), length_table)
log_header_line("Resuming from checkpoint: {}".format(True if resume else False), length_table)
log_separating_line(length_table)

rows = []
for seed in SEEDS:
    for strategy in STRATEGIES:
                
        np.random.seed(seed)
        torch.manual_seed(seed)
        if device == 'cuda':
            torch.cuda.manual_seed_all(seed)
             
        ### Model
        net = ResNet18()
        net = net.to(device)
        criterion = nn.CrossEntropyLoss # no function!
        optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
        
        ### load the data
        # if needed, specify batch sizes and shuffle settings
        dataloader = DataLoader(batch_size=batch_size)
        dataloader.download_cifar()
        dataloader.set_model(net)
        
        if strategy in ['max_k_loss', 'min_k_loss']:
                dataloader.initialize_weights(criterion(reduction='none'), device, seed=seed, 
                                              dump='./dump_logs/{}_{}_{}.txt'.format(strategy, seed, 0))
        if strategy in ['heterogeneous_max_k_loss', 'heterogeneous_min_k_loss']:
            dataloader.initialize_weights_per_class(criterion(reduction='none'), device, seed=seed)
            
        print()
        log_separating_line(length_table)
        log_position_header(seed, strategy, length_table)
        log_separating_line(length_table)

        if resume:
            assert os.path.isdir('serialized'), 'Error: no serialized directory found!'
            ckpt = torch.load('./serialized/{}/{}_ckpt_{}.pth'.format(given_date, strategy, seed))
            test_acc, start_epoch, net = net.load(ckpt)

        for epoch in range(start_epoch, start_epoch+num_epochs):
            #if strategy in ['max_k_loss', 'min_k_loss']:
            #    dataloader.initialize_weights(criterion(reduction='none'), device, seed=seed, 
            #                                  dump='./dump_logs/{}_{}_{}.txt'.format(strategy, seed, epoch))
            #if strategy in ['heterogeneous_max_k_loss', 'heterogeneous_min_k_loss']:
            #    dataloader.initialize_weights_per_class(criterion(reduction='none'), device, seed=seed)

            train_acc, train_loss = train(epoch, optimizer, criterion, seed, dataloader, \
                                          strategy, device)
            test_acc, test_loss = test(epoch, test_acc, criterion, seed, dataloader)
            log_position_line(epoch + 1, num_epochs, train_acc, test_acc, train_loss, test_loss, length_table)
            row = {
                'epoch': epoch + 1,
                'seed': seed,
                'train': True,
                'strategy': strategy,
                'accuracy': train_acc,
                'loss': train_loss
            }
            rows.append(row)
            row = {
                'epoch': epoch + 1,
                'seed': seed,
                'train': False,
                'strategy': strategy,
                'accuracy': test_acc,
                'loss': test_loss
            }
            rows.append(row)
        log_separating_line(length_table)
            
stop = timeit.default_timer()
time_needed = stop - start
hrs = int(time_needed / 3600)
mins = int((time_needed / 60) % 60)
secs = int(time_needed % 60)
print()
print("Finished training. Time needed: {} hrs {} mins {} secs".format(hrs, mins, secs))

logging_df = pd.DataFrame(rows, columns=['epoch', 'seed', 'train', 'strategy', 'accuracy', 'loss'])   
training_logs_dir = 'evaluation_logs'
logging_df.to_csv('noupdate_{}.txt'.format(os.path.join(training_logs_dir, today)), sep='\t', index=False)

Begin training.
+----------------------------------------------------------------------------------------+
| Seeds: [10, 42, 4]                                                                     |
| Strategies: ['max_k_loss', 'min_k_loss', 'heterogeneous_max_k_loss', 'heterogeneous_min_k_loss']|
| -> Resulting number of iterations: 12                                                  |
| Number of iterations: 100                                                              |
| Learning rate: 0.01                                                                    |
| Resuming from checkpoint: False                                                        |
+----------------------------------------------------------------------------------------+
Files already downloaded and verified
Files already downloaded and verified

+----------------------------------------------------------------------------------------+
| Seed: 10      Strategy: max_k_loss                                           

| [078/100]:    36.75           34.11           0.03426      0.03969                     |
| [079/100]:    36.11           33.76           0.03497      0.04022                     |
| [080/100]:    35.54           33.51           0.03563      00.0407                     |
| [081/100]:    34.99           33.11           0.03622      00.0411                     |
| [082/100]:    34.56           32.74           0.03676      0.04147                     |
| [083/100]:    34.09           32.47           0.03727      0.04181                     |
| [084/100]:    33.64           32.27           0.03774      0.04211                     |
| [085/100]:    033.2           31.96           0.03817      0.04238                     |
| [086/100]:    32.84           31.75           0.03857      0.04261                     |
| [087/100]:    32.46           31.39           0.03894      0.04284                     |
| [088/100]:    32.07           30.97           0.03928      0.04304                     |

| [063/100]:    010.0           010.0           00.0543      0.05588                     |
| [064/100]:    010.0           010.0           0.05417      0.05576                     |
| [065/100]:    010.0           010.0           0.05406      0.05564                     |
| [066/100]:    010.0           010.0           0.05395      0.05553                     |
| [067/100]:    010.0           010.0           0.05386      0.05544                     |
| [068/100]:    010.0           010.0           0.05378      0.05536                     |
| [069/100]:    010.0           010.0           0.05371      00.0553                     |
| [070/100]:    010.0           010.0           0.05366      0.05525                     |
| [071/100]:    010.0           010.0           0.05361      0.05521                     |
| [072/100]:    010.0           010.0           0.05357      0.05519                     |
| [073/100]:    010.0           010.0           0.05355      0.05518                     |

| [047/100]:    020.2           22.67           0.03719      0.03892                     |
| [048/100]:    20.19           22.67           0.03721      0.03892                     |
| [049/100]:    20.18           22.53           0.03722      00.0389                     |
| [050/100]:    20.16           22.47           0.03722      0.03888                     |
| [051/100]:    20.13           22.53           0.03723      0.03886                     |
| [052/100]:    20.07           22.48           0.03722      0.03882                     |
| [053/100]:    20.04           22.57           0.03721      0.03878                     |
| [054/100]:    20.01           22.68           00.0372      0.03873                     |
| [055/100]:    20.01           22.73           0.03718      0.03868                     |
| [056/100]:    19.99           22.68           0.03716      0.03863                     |
| [057/100]:    19.95           22.63           0.03712      0.03856                     |

| [031/100]:    020.5           22.69           0.03613      0.03794                     |
| [032/100]:    20.47           22.73           0.03625      0.03809                     |
| [033/100]:    20.43           022.6           0.03636      0.03823                     |
| [034/100]:    20.48           22.55           0.03646      0.03835                     |
| [035/100]:    20.42           22.51           0.03656      0.03845                     |
| [036/100]:    20.44           22.56           0.03665      0.03855                     |
| [037/100]:    20.46           22.58           0.03673      0.03863                     |
| [038/100]:    20.43           022.7           00.0368      00.0387                     |
| [039/100]:    20.35           22.66           0.03687      0.03875                     |
| [040/100]:    20.34           22.66           0.03693      00.0388                     |
| [041/100]:    20.26           22.75           0.03698      0.03884                     |

...

| [030/100]:    010.0           010.0           0.08375      0.06814                     |
| [031/100]:    010.0           010.0           0.08097      0.06589                     |
| [032/100]:    010.0           010.0           0.07847      0.06387                     |
| [033/100]:    010.0           010.0           0.07624      0.06209                     |
| [034/100]:    010.0           010.0           0.07427      0.06051                     |
| [035/100]:    010.0           010.0           0.07252      0.05912                     |
| [036/100]:    010.0           010.0           0.07098      00.0579                     |
| [037/100]:    010.0           010.0           0.06962      0.05683                     |
| [038/100]:    010.0           010.0           0.06843      00.0559                     |
| [039/100]:    010.0           010.0           00.0674      0.05509                     |
| [040/100]:    010.0           010.0           0.06649      0.05439                     |

| [014/100]:    018.8           18.97           0.03421      0.03533                     |
| [015/100]:    18.82           19.09           0.03434      00.0355                     |
| [016/100]:    18.83           19.17           0.03448      0.03568                     |
| [017/100]:    18.89           19.24           0.03463      0.03587                     |
| [018/100]:    18.91           19.35           0.03478      0.03606                     |
| [019/100]:    18.95           019.5           0.03494      0.03626                     |
| [020/100]:    18.94           19.56           0.03511      0.03645                     |
| [021/100]:    18.93           019.5           0.03527      0.03664                     |
| [022/100]:    18.96           19.53           0.03543      0.03683                     |
| [023/100]:    18.97           19.65           0.03558      0.03702                     |
| [024/100]:    18.97           19.63           0.03574      0.03719                     |

| [001/100]:    18.73           018.5           0.03377      0.03465                     |
| [002/100]:    18.73           18.62           0.03385      0.03472                     |
| [003/100]:    18.68           18.77           0.03381      0.03469                     |
| [004/100]:    18.69           18.79           0.03375      0.03466                     |
| [005/100]:    18.68           18.89           0.03371      0.03461                     |
| [006/100]:    18.67           18.95           0.03368      00.0346                     |
| [007/100]:    18.66           18.96           0.03368      00.0346                     |
| [008/100]:    18.67           19.03           00.0337      0.03464                     |
| [009/100]:    18.71           19.01           0.03374      0.03472                     |
| [010/100]:    18.72           18.94           0.03381      0.03481                     |
| [011/100]:    18.79           18.92           0.03389      0.03492                     |

| [092/100]:    17.46           18.48           0.03717      0.03689                     |
| [093/100]:    17.48           18.44           0.03723      0.03693                     |
| [094/100]:    17.51           18.48           00.0373      0.03698                     |
| [095/100]:    17.56           18.55           0.03738      0.03703                     |
| [096/100]:    17.64           18.73           0.03747      0.03709                     |
| [097/100]:    17.74           18.82           0.03755      0.03713                     |
| [098/100]:    17.77           19.03           0.03764      00.0372                     |
| [099/100]:    17.97           19.08           0.03773      0.03724                     |
| [100/100]:    18.11           019.2           0.03783      0.03732                     |
+----------------------------------------------------------------------------------------+

Finished training. Time needed: 48 hrs 0 mins 15 secs


FileNotFoundError: [Errno 2] No such file or directory: 'noupdate_evaluation_logs\\20200125.txt'

In [8]:
logging_df = pd.DataFrame(rows, columns=['epoch', 'seed', 'train', 'strategy', 'accuracy', 'loss'])   
training_logs_dir = 'evaluation_logs'
logging_df.to_csv('{}.txt'.format(os.path.join(training_logs_dir, 'noupdate', today)), sep='\t', index=False)